 ## We benchmark the neural network to the same dataset that we used for our experiments

 #### Load neural network

In [6]:
import torch
from eq_learner.architectures.naive_neural_network import ThreeLayerNet
from eq_learner.processing.tokenization import get_string
from eq_learner.evaluation import eqs_dataset_finder

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
models = ThreeLayerNet(1, 1000, 1).cuda().float()


#### Set Supports

In [7]:
import numpy as np
interpolation = np.arange(0.1,3,0.1)
extrapolation = np.arange(3,6,0.1)

 ### Load datasets

In [3]:
import torch
from eq_learner.evaluation import eqs_dataset_finder
import os
os.chdir("/home/labuser/repos/New-EQ-learn")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seen_eq = torch.load('data/1000_train.pt')
novel_eq = torch.load('data/1000_val.pt')

seen_train, seen_val = eqs_dataset_finder.convert_dataset_to_neural_network(seen_eq,interpolation,extrapolation)
novel_train, novel_val = eqs_dataset_finder.convert_dataset_to_neural_network(novel_eq, interpolation,extrapolation)

<string>:2: RuntimeWarning: invalid value encountered in log
<string>:2: RuntimeWarning: invalid value encountered in log
<string>:2: RuntimeWarning: invalid value encountered in log
<string>:2: RuntimeWarning: invalid value encountered in log
<string>:2: RuntimeWarning: invalid value encountered in log
<string>:2: RuntimeWarning: invalid value encountered in log
<string>:2: RuntimeWarning: invalid value encountered in log
<string>:2: RuntimeWarning: invalid value encountered in log
<string>:2: RuntimeWarning: invalid value encountered in log
<string>:2: RuntimeWarning: invalid value encountered in log
<string>:2: RuntimeWarning: invalid value encountered in log
<string>:2: RuntimeWarning: invalid value encountered in log
<string>:2: RuntimeWarning: invalid value encountered in log
<string>:2: RuntimeWarning: invalid value encountered in log
<string>:2: RuntimeWarning: invalid value encountered in log
<string>:2: RuntimeWarning: invalid value encountered in log
<string>:2: RuntimeWarni

#### We will train for each equation separately in the interpolation setting

In [4]:

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(models.parameters(), lr=1e-4)
for t in range(50000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = models(torch.tensor(interpolation).cuda().float()
)

    # Compute and print loss
    loss = criterion(y_pred, seen_train.cuda().float())
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

RuntimeError: size mismatch, m1: [1 x 29], m2: [1 x 1000] at /pytorch/aten/src/THC/generic/THCTensorMathBlas.cu:283

### Seen Equation benchmarking


In [20]:
ls = dict()
ls["rmse_test"] = list()
ls["rmse_train"] = list()
ls["gt_expressions"] = list()
for i in range(len(seen_val)):
    print('Iteration number: ', i)
    models = ThreeLayerNet(1, 1000, 1).cuda().float()
    criterion = torch.nn.MSELoss(reduction='sum')
    optimizer = torch.optim.Adam(models.parameters(), lr=1e-4)
    for t in range(10000):
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = models(torch.tensor(np.expand_dims(interpolation,1)).cuda().float())

        # Compute and print loss
        loss = criterion(y_pred, seen_train[i].unsqueeze(1).cuda().float())
        if t % 1000 == 99:
            print(t, loss.item())

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    models.eval()
    with torch.no_grad():
        #if any(np.isnan(novel_val[i].numpy())):
        #    continue
        pred = models(torch.tensor(np.expand_dims(extrapolation,1)).cuda().float())
        print(np.sqrt(np.mean(np.square(pred.cpu().squeeze().numpy()-seen_val[i].numpy()))))
        ls["rmse_test"].append(np.sqrt(np.mean(np.square(pred.cpu().squeeze().numpy()-seen_val[i].numpy()))))
        ls["rmse_train"].append(loss.cpu().numpy())
        ls["gt_expressions"].append(get_string(eqs_dataset_finder.remove_padding(seen_eq.tensors[1][0].numpy())))
        print(ls["rmse_test"][-1])
        print(ls["rmse_train"][-1])
        print(ls["gt_expressions"][-1])
        #print('Less than 0.20: ', np.sum(np.array(rmse_test)<0.20)/len(rmse_test))
        #print('Less than 0.10: ', np.sum(np.array(rmse_test)<0.10)/len(rmse_test))
        #print('Less than 0.05: ', np.sum(np.array(rmse_test)<0.05)/len(rmse_test))
    print()

Iteration number:  0
99 1.2485343217849731
1099 0.08370750397443771
2099 0.029527924954891205
3099 0.008345520123839378
4099 0.0025091078132390976
5099 0.0015803577844053507
6099 0.0014103322755545378
7099 0.0013307882472872734
8099 0.0012529653031378984
9099 0.0011318824253976345
0.3933784163518412
0.3933784163518412
0.0010338603
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  1
99 0.131094291806221
1099 0.00032209104392677546
2099 3.831236972473562e-05
3099 1.0935967111436184e-05
4099 3.871899934893008e-06
5099 1.5679941043345025e-06
6099 7.361473990386003e-07
7099 3.517645552619797e-07
8099 1.611662128198077e-07
9099 9.278964796521905e-08
0.11156111251615329
0.11156111251615329
9.5963514e-08
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  2
99 0.32745328545570374
1099 0.005836298689246178
2099 0.002093473682180047
3099 0.0008838856592774391
4099 0.0005384692922234535
5099 0.0003777768579311669
6099 0.0002754032320808619
7

1099 6.168472464196384e-05
2099 1.2655238606384955e-05
3099 7.72369276091922e-06
4099 4.424495273269713e-06
5099 2.0095112631679513e-06
6099 7.231707286337041e-07
7099 3.056535149426054e-07
8099 1.890138605631364e-07
9099 1.4204350406998856e-07
0.011236215281165918
0.011236215281165918
1.3582918e-07
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  22
99 0.002891690004616976
1099 3.602503420552239e-05
2099 1.0149102308787405e-05
3099 0.0013273628428578377
4099 1.1318602446408477e-06
5099 4.854840653933934e-07
6099 2.3015491024125367e-07
7099 1.1968117519245425e-07
8099 6.126951035412276e-08
9099 3.021594352503598e-08
0.03733115644401954
0.03733115644401954
1.8032175e-08
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  23
99 1.7335740327835083
1099 0.07290491461753845
2099 0.022239822894334793
3099 0.006430839188396931
4099 0.0020733356941491365
5099 0.0014087094459682703
6099 0.0012701046653091908
7099 0.0011383086675778031
809

1099 6.920543091837317e-05
2099 2.850013697752729e-05
3099 1.4671414646727499e-05
4099 8.867493306752294e-06
5099 5.340135885489872e-06
6099 3.1876634238869883e-06
7099 1.8644125248101773e-06
8099 1.0358495501350262e-06
9099 6.475862051047443e-07
0.026670991664652918
0.026670991664652918
4.4475584e-07
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  43
99 1.2489593029022217
1099 0.0831887423992157
2099 0.020895805209875107
3099 0.006356642581522465
4099 0.0023196579422801733
5099 0.0009581728372722864
6099 0.0003940714232157916
7099 0.0002006047434406355
8099 8.078847167780623e-05
9099 0.0013341272715479136
nan
nan
5.134693e-06
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  44
99 0.07843576371669769
1099 0.0009865338215604424
2099 3.92507208744064e-05
3099 1.1139849448227324e-05
4099 4.161075594311114e-06
5099 1.7574391222296981e-06
6099 9.927065320880502e-07
7099 6.457117933678092e-07
8099 4.6416494114964735e-07
9099 3.5598

3099 0.0002671529073268175
4099 3.103692506556399e-05
5099 1.3405660865828395e-05
6099 4.184520548733417e-06
7099 8.840170266921632e-07
8099 1.6721847373446508e-07
9099 2.8126814868301153e-05
0.11752722142766589
0.11752722142766589
3.2022275e-08
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  64
99 0.2416524738073349
1099 0.011720150709152222
2099 0.002265065675601363
3099 0.000640160811599344
4099 0.00022584060207009315
5099 8.22052825242281e-05
6099 4.0449987864121795e-05
7099 6.450356886489317e-05
8099 5.194182449486107e-06
9099 1.109871618609759e-06
10.591363458929202
10.591363458929202
1.0573056e-06
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  65
99 1.0196640491485596
1099 0.013769987970590591
2099 0.009334736503660679
3099 0.0063050673343241215
4099 0.0032946730498224497
5099 0.0014161872677505016
6099 0.0006367285386659205
7099 0.00031456814031116664
8099 0.00018843413272406906
9099 0.0001251216745004058
1.25087553

5099 2.1784603595733643
6099 2.0937564373016357
7099 2.0097906589508057
8099 1.9283759593963623
9099 1.8512661457061768
1.0655000319895354
1.0655000319895354
1.786432
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  85
99 0.9350850582122803
1099 0.04362538456916809
2099 0.013086211867630482
3099 0.003483663545921445
4099 0.001170509960502386
5099 0.0007379038725048304
6099 0.0006606804672628641
7099 0.0005834532203152776
8099 0.0003457956190686673
9099 0.00028470862889662385
0.19856064785739916
0.19856064785739916
0.0012942145
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  86
99 0.06939024478197098
1099 0.00016992897144518793
2099 2.9472303140209988e-05
3099 8.734228686080314e-06
4099 2.802954440994654e-06
5099 1.064654384208552e-06
6099 3.8192933971004095e-07
7099 1.1090442342265305e-07
8099 3.104742418713613e-08
9099 1.7167144505947363e-07
0.0524153249863146
0.0524153249863146
2.6157765e-09
log(exp(3*x)+exp(2*x)+1)+exp(sin

...

1099 0.015555737540125847
2099 0.0072408923879265785
3099 0.0036439704708755016
4099 0.0015236763283610344
5099 0.0005424245027825236
6099 0.00019420839089434594
7099 6.866647163406014e-05
8099 2.7261827199254185e-05
9099 0.00015296154015231878
0.06553603159680899
0.06553603159680899
0.00056469813
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  903
99 0.5145147442817688
1099 0.08995424956083298
2099 0.02106822282075882
3099 0.015526389703154564
4099 0.01492843497544527
5099 0.01454499177634716
6099 0.013978332281112671
7099 0.01304690632969141
8099 0.011933173052966595
9099 0.01065038237720728
1.3480582359388942
1.3480582359388942
0.00966722
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  904
99 0.3846415579319
1099 0.29572534561157227
2099 0.2602444291114807
3099 0.2284761518239975
4099 0.20105664432048798
5099 0.1776442676782608
6099 0.1588212251663208
7099 0.1477453112602234
8099 0.13930830359458923
9099 0.132973775267601

1099 0.008686973713338375
2099 0.0014979783445596695
3099 0.0004468587867449969
4099 0.00016926425450947136
5099 8.953009819379076e-05
6099 3.762152118724771e-05
7099 1.2340546163613908e-05
8099 0.0018554150592535734
9099 1.4986933820182458e-05
20.072754323320886
20.072754323320886
3.047103e-08
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  924
99 0.9646692276000977
1099 0.0018166648223996162
2099 0.001085714902728796
3099 0.0008660356397740543
4099 0.0006742461700923741
5099 0.0004905569367110729
6099 0.0003205858520232141
7099 0.00017582543659955263
8099 8.533109212294221e-05
9099 0.00011691343388520181
1.655708561578053
1.655708561578053
2.777743e-05
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  925
99 2.1527652740478516
1099 0.7632206082344055
2099 0.36160483956336975
3099 0.23464255034923553
4099 0.1964227557182312
5099 0.18529097735881805
6099 0.1819474697113037
7099 0.17978449165821075
8099 0.17762207984924316
9099

1099 0.14769524335861206
2099 0.12055709213018417
3099 0.10349263995885849
4099 0.09003230929374695
5099 0.07685591280460358
6099 0.06476997584104538
7099 0.054759398102760315
8099 0.04802956432104111
9099 0.04346241056919098
0.7144608306209128
0.7144608306209128
0.040442258
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  945
99 1.7402162551879883
1099 0.5307254791259766
2099 0.26962774991989136
3099 0.23827022314071655
4099 0.2291773557662964
5099 0.2217208445072174
6099 0.21400539577007294
7099 0.206107035279274
8099 0.1980927437543869
9099 0.189991295337677
1.67726243637073
1.67726243637073
0.18265942
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  946
99 1.4116687774658203
1099 0.010474488139152527
2099 0.003069077618420124
3099 0.0014642414171248674
4099 0.0006165156373754144
5099 0.00025251571787521243
6099 0.00010679140541469678
7099 4.560570960165933e-05
8099 2.209495687566232e-05
9099 1.3498635780706536e-05
0.973969

2099 0.00592718506231904
3099 0.0003669568686746061
4099 7.94385232438799e-06
5099 9.02767510524427e-07
6099 1.60732383847062e-07
7099 4.3695173701507883e-08
8099 1.1332486948845144e-08
9099 4.725178470721403e-09
0.007031496427731664
0.007031496427731664
1.5498683e-09
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  966
99 0.24920754134655
1099 0.010976344347000122
2099 0.0019441400654613972
3099 0.0005655275308527052
4099 0.0001982247194973752
5099 9.855131793301553e-05
6099 4.097581404494122e-05
7099 0.003318703267723322
8099 2.5320653094240697e-06
9099 6.532728775709984e-07
19.978890756920762
19.978890756920762
2.020944e-07
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  967
99 2.1737825870513916
1099 1.9573616981506348
2099 1.775665044784546
3099 1.592625617980957
4099 1.4258164167404175
5099 1.2754085063934326
6099 1.143913745880127
7099 1.042792558670044
8099 0.9652360677719116
9099 0.9027265310287476
1.1658916175388947

1099 0.3562443256378174
2099 0.29529035091400146
3099 0.26078832149505615
4099 0.23433849215507507
5099 0.20882409811019897
6099 0.18592777848243713
7099 0.16437944769859314
8099 0.14432117342948914
9099 0.12988021969795227
1.3143004878904883
1.3143004878904883
0.12010475
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  987
99 2.0337400436401367
1099 1.4409239292144775
2099 1.1448304653167725
3099 0.918440043926239
4099 0.7714700102806091
5099 0.6633317470550537
6099 0.6077359318733215
7099 0.569067120552063
8099 0.5385177135467529
9099 0.515578031539917
3.2750363806019913
3.2750363806019913
0.4991602
log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)**2+sin(x)+1)

Iteration number:  988
99 0.58200603723526
1099 0.14870741963386536
2099 0.029052240774035454
3099 0.0019367409404367208
4099 7.728046330157667e-05
5099 3.71550559066236e-05
6099 1.7486703654867597e-05
7099 1.2378581232042052e-05
8099 1.0331682460673619e-05
9099 8.96704204933485e-06
0.099025336831034

In [21]:
import pandas as pd
df_eval_neural_network = pd.DataFrame(ls)
df_eval_neural_network.head()
df_eval_neural_network.to_pickle("df_seen_neural_network")

In [24]:
df_eval_neural_network

,rmse_test,rmse_train,gt_expressions
0,0.393378,0.0010338603,log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)*...
1,0.111561,9.5963514e-08,log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)*...
2,0.826458,0.0004423303,log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)*...
3,0.137196,4.1840536e-08,log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)*...
4,1.816706,0.7261362,log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)*...
...,...,...,...
995,0.491843,0.00053261477,log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)*...
996,1.370835,0.011405404,log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)*...
997,NaN,1.8851235e-06,log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)*...
998,NaN,1.7995653e-05,log(exp(3*x)+exp(2*x)+1)+exp(sin(x)**3+sin(x)*...


In [18]:
import pandas as pd
df_eval_neural_network = pd.DataFrame(l)
df_eval_neural_network.head()
df_eval_neural_network.to_pickle("df_seen_neural_network")

{'rmse_test': [360.1967490416201,
  0.025609926063604894,
  nan,
  363.9046683286592,
  126.11397004073638,
  0.2352097831839418,
  1.2756478039180168,
  0.29694697782680185,
  0.17010267250822725,
  0.1261180717005567,
  1.9884625108733682,
  0.17419020571895788,
  22.470271094709837,
  9.296021082438344,
  0.1886465490488819,
  0.13752684617257716,
  nan,
  0.5336963989894047,
  0.20789742243843287,
  0.23919097428984698,
  0.10260919050913124,
  nan,
  21.256125972561648,
  314.7947547984665,
  0.07834918789340105,
  5.437655360376135,
  0.0810960558219033,
  0.6706570227420265,
  0.007335157075909736,
  21.24521155917999,
  0.5621846693176554,
  0.5635896759856603,
  244.4873010879843,
  0.42304257198612644,
  0.4668755517748036,
  0.5123575145889352,
  0.10839701755853515,
  19.928342246889507,
  360.6455191530352,
  19.594951956912578,
  0.03496086821852845,
  0.21441599951809195,
  362.82370065649536,
  nan,
  1.2758554715381833,
  24.093831947272726,
  0.6083156514192296,
  9.3

### Unseen equation benchmarking

In [ ]:
l = dict()
l["rmse_test"] = list()
l["rmse_train"] = list()
l["gt_expressions"] = list()
for i in range(len(seen_val)):
    print('Iteration number: ', i)
    models = ThreeLayerNet(1, 1000, 1).cuda().float()
    criterion = torch.nn.MSELoss(reduction='sum')
    optimizer = torch.optim.Adam(models.parameters(), lr=1e-4)
    for t in range(10000):
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = models(torch.tensor(np.expand_dims(interpolation,1)).cuda().float())

        # Compute and print loss
        loss = criterion(y_pred, novel_train[i].unsqueeze(1).cuda().float())
        if t % 1000 == 99:
            print(t, loss.item())

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    models.eval()
    with torch.no_grad():
        #if any(np.isnan(novel_val[i].numpy())):
        #    continue
        pred = models(torch.tensor(np.expand_dims(extrapolation,1)).cuda().float())
        print(np.sqrt(np.mean(np.square(pred.cpu().squeeze().numpy()-novel_val[i].numpy()))))
        l["rmse_test"].append(np.sqrt(np.mean(np.square(pred.cpu().squeeze().numpy()-novel_val[i].numpy()))))
        l["rmse_train"].append(loss.cpu().numpy())
        l["gt_expressions"].append(get_string(eqs_dataset_finder.remove_padding(novel_eq.tensors[1][0].numpy())))
        print(l["rmse_test"][-1])
        print(l["rmse_train"][-1])
        print(l["gt_expressions"][-1])
        #print('Less than 0.20: ', np.sum(np.array(rmse_test)<0.20)/len(rmse_test))
        #print('Less than 0.10: ', np.sum(np.array(rmse_test)<0.10)/len(rmse_test))
        #print('Less than 0.05: ', np.sum(np.array(rmse_test)<0.05)/len(rmse_test))
    print()